In [1]:
import sys
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt



path='/home/jbohne/sbu/'
sys.path.append(f'{path}taq_db/TAQ-Query-Scripts/data')
sys.path.append(f'{path}research/Adaptive_Signal_Estimation_Private')

from trend_filtering.adaptive_tf import adaptive_tf
from trend_filtering.test_adaptive_tf import test_adaptive_tf
from profiling.profiler import profile

In [2]:
trades=pd.read_csv(f'{path}taq_db/TAQ-Query-Scripts/data/AAPL_trades.csv',index_col=0).dropna()

price=trades['Trade_Price'].values

In [3]:
price.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

### Cross Validation is 10 fold + OOS Fit

In [4]:


profile(test_adaptive_tf,None)



Duality Gap is 48.79813242461838
Duality Gap is 48.79813023204345
Duality Gap is 264.27474517365596
Duality Gap is 263.982935119745
Duality Gap is 263.9787163933342
Duality Gap is 263.97862348495596
Duality Gap is 263.97837846655295
Duality Gap is 263.97825540911754
Duality Gap is 263.9781938139769
Duality Gap is 263.97816300866066
Duality Gap is 263.9781476051681
Duality Gap is 263.9781399033502
Duality Gap is 263.97813605244045
Duality Gap is 263.97813412698736
Duality Gap is 263.9781331642615
Duality Gap is 263.9781326828988
Duality Gap is 260.0500769739499
Duality Gap is 260.05007697367387
Duality Gap is 260.0500769597044
Duality Gap is 260.0500769597044
Duality Gap is 260.04989576783896
Duality Gap is 260.0498051215563
Duality Gap is 260.04975979237884
Duality Gap is 260.0497371271071
Duality Gap is 260.0497257944038
Duality Gap is 260.049720128048
         34338 function calls (33294 primitive calls) in 0.302 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  c